# ANOVA

## Objectives
- Conduct an ANOVA test to compare the means of many populations.

## Introduction to ANOVA

ANOVA stands for ANalysis Of VAriance. An ANOVA test is a hypothesis test used to compare the means among several populations. The test uses variances to help determine if the population means are equal or not. To perform a one-way ANOVA test, the following basic assumptions must be fulfilled:

- Each population is normally distributed.
- All samples are randomly selected and independent.
- The populations have equal variances (or standard deviations).

The null hypothesis is that all the population means are the same. The alternative hypothesis is that at least one pair of means is different. For example, if there are $k$ populations:

\begin{align*}
&H_0:\mu_1 = \mu_2 = \mu_3 = \cdots = \mu_k \\
&H_a: \text{At least one mean isn't equal to all the other means}
\end{align*}

ANOVA works by comparing the approximate variance *between* the samples (that is, how much the samples vary with each other) with the approximate variance *within* the samples (that is, how much on average each sample varies on its own). If the null hypothesis is true so that the population means are all equal, then the variance between the samples and the variance within the samples should be about equal.

In [5]:
#**ALT=Two groups of box plots. Group A has little variation between box plots; group B has significant variation between box plots.**#
png("anova_illustration.png", width = 1000, height = 500)

library(repr)
options(repr.plot.width = 7, repr.plot.height=4)
par(mfrow = c(1, 2))

n <- 500

A <- rnorm(n, 0, 5)
B <- rnorm(n, 0, 5)
C <- rnorm(n, 0, 5)
D <- rnorm(n, 0, 5)
boxplot(A, B, C, D, col="#20639b", axes=FALSE, outline=FALSE, main= "A", lwd = 3, cex.main = 3)

A <- rnorm(n, 4, 5)
B <- rnorm(n, 0, 5)
C <- rnorm(n, 11, 5)
D <- rnorm(n, -2, 5)
boxplot(A, B, C, D, col = "#ee5c42", axes = FALSE, outline=FALSE, main = "B", lwd = 3, cex.main = 3)

dev.off()

png 
  2

```{figure} anova_illustration.png
---
width: 100%
alt: Two groups of box plots are pictures, with four box plots in group A and four box plots in group B. There is little variation between the box plots in group A. There is significant variation between the box plots in group B.
name: anova-illustration
---
The four box plots in group A represent four populations. There is little variation between the box plots in group A. It might be reasonable to expect the populations in group A to all have the same mean. Compare this to the four populations represented by the four box plots in group B. There is significant variation between the box plots in group B. We might expect the populations in group B to have different means.
```

To better understand the principles behind ANOVA, consider the two groups of populations represented by the box plots in {numref}`Figure {number} <anova-illustration>`. In group A, notice that there is little variation between the box plots of the different samples. It is reasonable to think that the underlying population means may be equal. 

In comparison, there is considerable variation in group B. In fact, there is so much variation between these samples that it is very unlikely that the underlying population means between these samples are all equal.

## Notation
An ANOVA test can involve several populations and samples. To make our meaning clear, we will use the following notation in this section.

- $x_j$ are the data values in the $j$th sample.
- $n_j$ is the sample size of the $j$th sample.
- $\bar{x}_j$ is the mean of the $j$th sample. As usual, the sample mean $\bar{x}_j$ is the sum of all data values in the sample divided by sample size:

$$\bar{x}_j = \dfrac{\sum x_j}{n_j}.$$

- $\bar{\bar{x}}$ is the **grand mean**. It is the mean of the combined data values of all the samples. It is calculated by adding all data values from all samples together, then dividing by the total number of all the data values from all samples:

$$\bar{\bar{x}} = \frac{\sum x_1 + \sum x_2 + \cdots + \sum x_k}{n_1 + n_2 + \cdots + n_k},$$

where $k$ is the total number of samples, where one sample is taken from each population.

A simple example may help to clarify the notation.

***

### Example 9.3.1
Three small samples were drawn from three populations. The sample data obtained is:

Sample $1$:  18, 21, 20, 20, 17 <br/>
Sample $2$:  22, 19, 18, 21, 19, 16, 18 <br/>
Sample $3$:  20, 19, 17, 21
    
The data values $x_1$ in sample $1$ are 18, 21, 20, 20, 17. There are $5$ data values in sample $1$, so $n_1 = 5$. We can use R to calculate the mean $\bar{x}_1$ of sample $1$.

In [1]:
x1 = c(18, 21, 20, 20, 17)
n1 = length(x1)

xbar1 = sum(x1)/n1
xbar1

[1] 19.2

The mean of sample $1$ is $\bar{x}_1 = 19.2$.

The data values $x_2$ in sample $2$ are 22, 19, 18, 21, 19, 16, 18. There are $7$ data values in sample $2$, so $n_2 = 7$. We can use R to calculate the mean $\bar{x}_2$ of sample $2$.

In [2]:
x2 = c(22, 19, 18, 21, 19, 16, 18)
n2 = length(x2)

xbar2 = sum(x2)/n2
xbar2

[1] 19

The mean of sample $2$ is $\bar{x}_2 = 19$.

The data values $x_3$ in sample $3$ are 20, 19, 17, 21. There are $4$ data values in sample $3$, so $n_3 = 4$. We can use R to calculate the mean $\bar{x}_3$ of sample $3$.

In [3]:
x3 = c(20, 19, 17, 21)
n3 = length(x3)

xbar3 = sum(x3)/n3
xbar3

[1] 19.25

The mean of sample $3$ is $\bar{x}_3 = 19.25$.

Now we can use R to calculate the grand mean $\bar{\bar{x}}$ of these three samples.

In [4]:
grandx = (sum(x1) + sum(x2) + sum(x3))/(n1 + n2 + n3)
grandx

[1] 19.125

The grand mean&mdash;the mean of all the data in all three samples&mdash;is $\bar{\bar{x}} = 19.125$.

***

## The Test Statistic For an ANOVA Test
To calculate the test statistic for an ANOVA test with $k$ samples (one sample from each population), we calculate two estimates of the variance.

The first estimate of the variance is $MST$, which is the **m**ean **s**quare estimate of the variance among **t**reatments. $MST$ is the variance *between* the sample means of the different samples. We calculate $MST$ using the formula

$$ MST = \frac{SST}{DFT}, $$

where $SST$, called the **s**um of **s**quares among **t**reatments, is the sum of squared differences between the sample means and the grand mean, and $DFT$ is the **d**egrees of **f**reedom for **t**reatments. $SST$ and $DFT$ are given by the formulas

\begin{align*}
SST &= n_1(\bar{x}_1 - \bar{\bar{x}})^2 + n_2(\bar{x}_2 - \bar{\bar{x}})^2 + \cdots + n_k(\bar{x}_k - \bar{\bar{x}})^2, \\
DFT &= k - 1.
\end{align*}

The second estimate of the variance is $MSE$, which is the **m**ean **s**quare estimate for **e**rrors. $MSE$ measures the variance *within* the samples. We calculate $MSE$ using the formula

$$ MSE = \frac{SSE}{DFE}, $$

where $SSE$, the **sum** of **s**quares for **e**rrors, is the total sum of squared differences between the data values of each sample and the sample's mean, and $DFE$ is the degrees of freedom for errors. We calculate $SSE$ and $DFE$ using the formulas

\begin{align*}
SSE &= \sum (x_1 - \bar{x}_1)^2 + \sum (x_2 - \bar{x}_2)^2 + \cdots + \sum (x_k - \bar{x}_k)^2, \\
DFE &= (n_1 - 1) + (n_2 - 1) + \cdots + (n_k - 1).
\end{align*}

$MST$ and $MSE$ both estimate variances, so we use an $F$-distribution to perform an ANOVA test just like we did when comparing two population variances. The $F$-statistic we use for an ANOVA test is given by

$$ F = \frac{MST}{MSE}. $$

We use an $F$-distribution with numerator degrees of freedom $DFT$ and denominator degrees of freedom $DFE$ to calculate the $p$-value.

If the null hypothesis is true (that is, if the means of all the populations are equal), then we would expect the variance between samples, $MST$, to be no larger than the average variance within a sample, $MSE$. In this case, 

$$F = \frac{MST}{MSE} \leq 1.$$

But if the null hypothesis is not true, then we would expect the variance between samples, $MST$, to be greater than the average variance within a sample, $MSE$, in which case 

$$F = \frac{MST}{MSE} > 1.$$

Because of this, an ANOVA test is always a right-tailed test.

The general steps for completing a hypothesis test have not changed:
1. State the null and alternative hypotheses.
2. Assuming the null hypothesis is true, identify the sampling distribution.
3. Find the $p$-value.
4. Draw a conclusion.

***

### Example 9.3.2

A meteorologist wishes to test if the average daily high temperature is the same for each of the past four years. She randomly selects a number of days from each year and records the daily high temperature on each selected day (in °F):

Year 1: <br/>
82, 104, 119, 56, 85, 94, 81, 106, 82, 92, 109, 71, 95, 86, 34, 89, 80, 53, 78, 99, 57, 87, 69, 98, 75, 88, 59, 104, 65, 66, 74, 73, 73, 106, 91, 89, 85, 84, 53, 81
    
Year 2: <br/>
80, 52, 72, 72, 43, 61, 66, 58, 118, 73, 76, 64, 81, 65, 63, 78, 72, 83, 104, 54, 69, 100, 63, 71, 38, 73, 92, 70, 86, 51, 66, 89, 74, 85

Year 3: <br/>
75, 87, 79, 96, 72, 75, 93, 84, 80, 95, 72, 56, 83, 98, 96, 54, 106, 66, 120, 77, 88, 90, 75, 98, 113, 55, 44

Year 4: <br/>
92, 68, 74, 82, 94, 44, 63, 61, 99, 71, 85, 42, 90, 94, 48, 97, 89, 65, 81, 45, 93, 81, 73, 69, 93, 75, 76, 61, 75, 102
    
Use an ANOVA test with a level of significance of $6\%$ to test if the average daily high temperature is the same for all four years.

#### Solution
##### Step 1: State the null and alternative hypotheses.
For an ANOVA test, the null hypothesis is always that the means of all the populations are equal. The alternative hypothesis for an ANOVA test is always that at least one of the population means is not equal to the others. That is,

\begin{align*}
H_0:&\ \mu_1 = \mu_2 = \mu_3 = \mu_4 \\
H_a:&\ \text{At least one mean isn't equal to the other means}
\end{align*}

##### Step 2: Assuming the null hypothesis is true, identify the sampling distribution.
To perform an ANOVA test, we use an $F$-distribution. The numerator degrees of freedom is the degrees of freedom for treatments, $DFT$. It is one less than the number of samples. In this example, we have $k = 4$ samples, so the numerator degrees of freedom is

$$ DFT = k - 1 = 4 - 1 = 3. $$

The denominator degrees of freedom is the degrees of freedom for errors, $DFE$. To calculate $DFE$, we first must know the size of each of the four samples. We will use R to count the size of each sample.

In [2]:
x1 = c(82, 104, 119, 56, 85, 94, 81, 106, 82, 92, 109, 71, 95, 86, 34, 89, 80, 53, 78, 99, 57, 87, 69, 98, 75, 88, 59, 104, 65, 66, 74, 73, 73, 106, 91, 89, 85, 84, 53, 81)
x2 = c(80, 52, 72, 72, 43, 61, 66, 58, 118, 73, 76, 64, 81, 65, 63, 78, 72, 83, 104, 54, 69, 100, 63, 71, 38, 73, 92, 70, 86, 51, 66, 89, 74, 85)
x3 = c(75, 87, 79, 96, 72, 75, 93, 84, 80, 95, 72, 56, 83, 98, 96, 54, 106, 66, 120, 77, 88, 90, 75, 98, 113, 55, 44)
x4 = c(92, 68, 74, 82, 94, 44, 63, 61, 99, 71, 85, 42, 90, 94, 48, 97, 89, 65, 81, 45, 93, 81, 73, 69, 93, 75, 76, 61, 75, 102)

n1 = length(x1)
n2 = length(x2)
n3 = length(x3)
n4 = length(x4)

n1
n2
n3
n4

[1] 40

[1] 34

[1] 27

[1] 30

So $n_1 = 40$, $n_2 = 34$, $n_3 = 27$, and $n_4 = 30$. Then the denominator degrees of freedom is

$$ DFE = (n_1 - 1) + (n_2 - 1) + (n_3 - 1) + (n_4 - 1) = (40 - 1) + (34 - 1) + (27 - 1) + (30 - 1) = 127.$$

##### Step 3: Find the $p$-value.

To find the test statistic, we first need to calculate the sample mean of each of the four samples as well as the grand mean $\bar{\bar{x}}$.

In [3]:
xbar1 = sum(x1)/n1
xbar1

xbar2 = sum(x2)/n2
xbar2

xbar3 = sum(x3)/n3
xbar3

xbar4 = sum(x4)/n4
xbar4

grandx = (sum(x1) + sum(x2) + sum(x3) + sum(x4))/(n1 + n2 + n3 + n4)
grandx

[1] 81.8

[1] 72.41176

[1] 82.48148

[1] 76.06667

[1] 78.19084

We find that $\bar{x}_1 = 81.800$, $\bar{x}_2 = 72.412$, $\bar{x}_3 = 82.481$, $\bar{x}_4 = 76.067$, and the grand mean is $ \bar{\bar{x}} = 78.191$.

We can now use these values to find 

$$SST = n_1(\bar{x}_1 - \bar{\bar{x}})^2 + n_2(\bar{x}_2 - \bar{\bar{x}})^2 + n_3(\bar{x}_3 - \bar{\bar{x}})^2 + n_4(\bar{x}_4 - \bar{\bar{x}})^2.$$ 

In [5]:
SST = n1*(xbar1 - grandx)^2 + n2*(xbar2 - grandx)^2 + n3*(xbar3 - grandx)^2 + n4*(xbar4 - grandx)^2
SST

[1] 2288.986

So $SST = 2288.986$. With $SST$ calculated and the value of $DFT$ found earlier, we are ready to find 

$$ MST = \frac{SST}{DFT}. $$

In [6]:
DFT = 3

MST = SST/DFT
MST

[1] 762.9954

Thus, $MST = 762.995$. This is the numerator of the test statistic.

We still need to find the denominator, $MSE$. To find $MSE$, we first calculate 

$$SSE = \sum (x_1 - \bar{x}_1)^2 + \sum (x_2 - \bar{x}_2)^2 + \sum (x_3 - \bar{x}_3)^2 + \sum(x_4 - \bar{x}_4)^2.$$

In [7]:
SSE = sum( (x1 - xbar1)^2 ) + sum( (x2 - xbar2)^2 ) + sum( (x3 - xbar3)^2 ) + sum( (x4 - xbar4)^2 )
SSE

[1] 38587.24

We find that $SSE = 38587.243$. Using this value for $SSE$ together with the value for $DFE$ we found earlier, we can calculate

$$ MSE = \frac{SSE}{DSE}. $$

In [8]:
DFE = 127

MSE = SSE/DFE
MSE

[1] 303.8366

So $MSE = 303.837$. This is the denominator value of the test statistic.

We now have all the pieces to calculate the test statistic,

$$ F = \frac{MST}{MSE}. $$

In [9]:
F = MST/MSE
F

[1] 2.511204

Because an ANOVA test involves so many calculations, statisticians often organize the data into a table like in {numref}`anova-table`.

```{list-table} A table summarizing the important calculations for the ANOVA test in this example.
:header-rows: 1
:stub-columns: 1
:name: anova-table
* -
  - SS
  - DF
  - MS
  - F
* - Treatments
  - 2288.986
  - 3
  - 762.995
  - 2.511
* - Errors
  - 38587.243
  - 127
  - 303.837
  -
```

Now that we have our test statistic, we are ready to find the $p$-value. Remember that an ANOVA test is always a right-tailed test.

In [10]:
1 - pf(q = F, df1 = DFT, df2 = DFE)

[1] 0.06161711

So the $p$-value is $P(F > 2.511) = 0.062$. That is, there is about a $6.2\%$ chance that we would randomly sample data with this much variation between samples relative to the variation within samples if the population means were all actually equal.

##### Step 4: Draw a conclusion.
Since the $p$-value $= 0.062 \geq 0.06 = \alpha$, we do *not* reject the null hypothesis. There is not enough evidence to conclude that the average daily high temperature for at least one year is different than average in the other years sampled.